In [2]:
from utils import load_dict_from_file, all_int_in_set, flatten
from utils_nlp import get_number_tokens
import numpy as np
import os
import pandas as pd
import re

from dotenv import load_dotenv

load_dotenv()

dataset_name = os.getenv("dataset_name")
data_path = os.getenv("data_path")

In [3]:
list_question_ids = [0, 2, 3, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 19, 20, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 69, 70, 71, 72, 73, 74, 75, 77, 78, 80, 81, 83, 84, 85, 86, 87, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 104, 105, 107, 108]
print(len(list_question_ids))

92


In [4]:
def get_dict_compiled_results(results_dict):
    list_precision = []
    list_recall = []
    list_answer_in_context = []
    list_len_context = []
    list_smallest_context_needed = []
    list_matching_reponse_id = []
    for item in results_dict.keys():
        item_as_int = int(item)
        if item_as_int in list_question_ids:
            list_precision_ = []
            list_recall_ = []
            list_answer_in_context_ = []
            list_len_context_ = []
            list_smallest_context_needed_ = []
            list_matching_reponse_id_ = []
            answer_id = set(results_dict[item]["answer_id_"])
            answer_dicts = [answer_ for answer_ in results_dict[item].keys() if type(results_dict[item][answer_]) == dict]

            for dict_name in answer_dicts:
                response_id = set(flatten(results_dict[item][dict_name]["matching_ids"]))
                response_name = set(flatten(results_dict[item][dict_name]["matching_ids"]))


                context_ = str(results_dict[item][dict_name]["context"]).lower()
                list_len_context_.append(get_number_tokens(context_))

                list_precision_.append(len(response_id.intersection(answer_id)) / len(response_id) if len(response_id) > 0 else 0)
                list_recall_.append(len(response_id.intersection(answer_id)) / len(answer_id) if len(answer_id) > 0 else 0)
                list_matching_reponse_id_.append(len(response_id)/len(response_name) if len(response_name) > 0 else 0)

                if item_as_int == 81:
                    print(response_id)
                    print(answer_id)
                    print(list_precision_)

                count_in_context = 0
                indx = 0

                for answer in results_dict[item]["answer_name"]:
                    if answer.lower() in context_:
                        count_in_context += 1
                        # find where answer in in context_
                        indx_ = context_.index(answer.lower())
                        if indx_ > indx:
                            indx = indx_
                    else:
                        indx = 999999

                if indx == 999999:
                    list_smallest_context_needed_.append(np.nan)
                else:
                    list_smallest_context_needed_.append(get_number_tokens(context_[:indx]))

                is_in_context = count_in_context/len(results_dict[item]["answer_name"])

                list_answer_in_context_.append(is_in_context)  

            list_precision.append(list_precision_)
            list_recall.append(list_recall_)
            list_answer_in_context.append(list_answer_in_context_)
            list_len_context.append(list_len_context_)
            list_smallest_context_needed.append(list_smallest_context_needed_)
            list_matching_reponse_id.append(list_matching_reponse_id_)

        else:
            pass
            # print(f"Question {item} not in list_question_ids")

    dict_ = {
        "precision": list_precision,
        "recall": list_recall,
        "answer_in_context": list_answer_in_context,
        "len_context": list_len_context,
        "smallest_context_needed": list_smallest_context_needed,
        "matching_reponse_id": list_matching_reponse_id
    } 
    return dict_


In [23]:
results_dict = load_dict_from_file(data_path + "04_09_RAG_v2.json")
results_rag = get_dict_compiled_results(results_dict)


for item in results_rag.items():
    print(item[0])
    print(np.nanmean(item[1]))

Dictionary loaded from ../data/results/04_09_RAG_v2.json
{62634}
{62640, 62634}
[1.0]
precision
0.328804347826087
recall
0.2748188405797101
answer_in_context
0.4298913043478261
len_context
7130.79347826087
smallest_context_needed
1064.4242424242425
matching_reponse_id
0.8152173913043478


In [20]:
results_dict = load_dict_from_file(data_path + "05_09_LINK_10k_max_score.json")
results_sp_10k = get_dict_compiled_results(results_dict)


for item in results_sp_10k.items():
    print(item[0])
    print(np.nanmean(item[1]))

Dictionary loaded from ../data/results/05_09_LINK_10k_max_score.json
{128033}
{62640, 62634}
[0.0]
precision
0.3076379606454093
recall
0.36086956521739133
answer_in_context
0.6958333333333333
len_context
9587.869565217392
smallest_context_needed
2542.4754098360654
matching_reponse_id
1.0


In [27]:
results_dict["42"]

{'question_': "I'm currently taking tremelimumab for HIV. Do you know of any antibody-like molecules that would work well with my current treatment?",
 'answer_id_': [18040, 19560, 18039],
 'answer_name': ['Leronlimab', 'PRO-542', 'CCR5 mAb'],
 'answer_0': {'response': ['Ibalizumab',
   'Emapalumab',
   'Ofatumumab',
   'Afelimomab',
   'Urelumab',
   'Alemtuzumab',
   'Iratumumab',
   'Adalimumab',
   'Leronlimab',
   'Tositumomab',
   'Elotuzumab',
   'Tafasitamab',
   'Daratumumab',
   'XmAb 2513',
   'Sarilumab',
   'Infliximab',
   'Olaratumab',
   'Ibritumomab tiuxetan',
   'Belimumab',
   'Utomilumab',
   'Zalutumumab',
   'Inebilizumab',
   'Ligelizumab',
   'Golimumab',
   'Sifalimumab',
   'Sulesomab',
   'Epratuzumab'],
  'context': 'Tremelimumab has a synergistic interaction with Ibalizumab is an indication for the treatment of AIDS.\nTremelimumab has a synergistic interaction with Ibalizumab is an indication for the treatment of HIV infectious disease.\nTremelimumab has a 

In [24]:
results_dict = load_dict_from_file(data_path + "06_09_BOTH_10k_max_text.json")
results_sp_10k_important = get_dict_compiled_results(results_dict)

for item in results_sp_10k_important.items():
    print(item[0])
    print(np.nanmean(item[1]))

Dictionary loaded from ../data/results/06_09_BOTH_10k_max_text.json
{128040}
{62640, 62634}
[0.0]
precision
0.2783724930464061
recall
0.3056159420289855
answer_in_context
0.6300724637681159
len_context
8356.326086956522
smallest_context_needed
2056.6792452830186
matching_reponse_id
0.9347826086956522


In [26]:
results_dict["42"]

{'question_': "I'm currently taking tremelimumab for HIV. Do you know of any antibody-like molecules that would work well with my current treatment?",
 'answer_id_': [18040, 19560, 18039],
 'answer_name': ['Leronlimab', 'PRO-542', 'CCR5 mAb'],
 'answer_0': {'response': ['Ibalizumab',
   'Emapalumab',
   'Ofatumumab',
   'Afelimomab',
   'Urelumab',
   'Alemtuzumab',
   'Iratumumab',
   'Adalimumab',
   'Leronlimab',
   'Tositumomab',
   'Elotuzumab',
   'Tafasitamab',
   'Daratumumab',
   'XmAb 2513',
   'Sarilumab',
   'Infliximab',
   'Olaratumab',
   'Ibritumomab tiuxetan',
   'Belimumab',
   'Utomilumab',
   'Zalutumumab',
   'Inebilizumab',
   'Ligelizumab',
   'Golimumab',
   'Sifalimumab',
   'Sulesomab',
   'Epratuzumab'],
  'context': 'Tremelimumab has a synergistic interaction with Ibalizumab is an indication for the treatment of AIDS.\nTremelimumab has a synergistic interaction with Ibalizumab is an indication for the treatment of HIV infectious disease.\nTremelimumab has a 

In [18]:
results_dict = load_dict_from_file(data_path + "05_09_BOTH_10k_tfidf_rank.json")
results_sp_10k_tfidf = get_dict_compiled_results(results_dict)

for item in results_sp_10k_tfidf.items():
    print(item[0])
    print(np.nanmean(item[1]))

Dictionary loaded from ../data/results/05_09_BOTH_10k_tfidf_rank.json
{49977}
{62640, 62634}
[0.0]
precision
0.30738048183700356
recall
0.29347826086956524
answer_in_context
0.6096014492753623
len_context
8597.184782608696
smallest_context_needed
2463.72
matching_reponse_id
0.9891304347826086


# By question

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def get_variable_name(variable):
    names = [name for name in globals() if globals()[name] is variable]
    if names:
        return names[0]
    return None

def plot_by_question(data, name_of_metric="metric", data2=None):
    if name_of_metric == "metric":
        name_of_metric = get_variable_name(data)


    # Plotting
    questions = np.array(list_question_ids)

    plt.figure(figsize=(14, 3))

    def plot_(data_, colors=["blue", "green"], marker='o'):
        # Calculate mean, min, and max for each question
        means = [np.nanmean(d) for d in data_]
        mins = [np.nanmin(d) if len(d) > 0 else np.nan for d in data_]
        maxs = [np.nanmax(d) if len(d) > 0 else np.nan for d in data_]
        lower_errors = np.clip([m_ - min_ for m_, min_ in zip(means, mins)], 0, 1)
        upper_errors = np.clip([max_ - m_ for m_, max_ in zip(means, maxs)], 0, 1)

        # Plot non-NaN points
        is_nan = np.isnan(means)
        plt.errorbar(questions[~is_nan], np.array(means)[~is_nan], yerr=[np.array(lower_errors)[~is_nan], np.array(upper_errors)[~is_nan]], 
                    fmt='o', capsize=5, marker=marker, markersize=5, color=colors[0])    

        if np.sum(is_nan) > 0:
            # Explicitly plot NaN points as green dots
            plt.scatter(questions[is_nan], np.zeros_like(questions[is_nan]), color=colors[1], marker=marker, s=50, label="NaN Points")

    plot_(data, colors=["blue", "green"], marker='o')
    if data2 is not None:
        plot_(data2, colors=["purple", "darkgreen"], marker='x')

    plt.xlabel('Question', fontsize=14)
    plt.ylabel(f"{name_of_metric}", fontsize=14)
    plt.title(f"Mean {name_of_metric} with Error Bars for Each Question", fontsize=16)
    plt.xticks(questions, rotation=90)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
plot_by_question(results_rag["precision"], "precision", results_sp_10k["precision"])

In [ ]:
plot_by_question(results_rag["answer_in_context"], "answer_in_context", results_sp_10k["answer_in_context"])

In [ ]:
plot_by_question(results_rag["smallest_context_needed"], "smallest_context_needed", results_sp_10k["smallest_context_needed"])

In [ ]:
plot_by_question(results_rag["len_context"], "len_context", results_sp_10k["len_context"])